#### Project size estimation

In [3]:
import re

def remove_comments_and_empty_lines_PY(code):
    # Remove single-line comments
    code = re.sub(r'#.*', '', code)
    # Remove multi-line comments
    code = re.sub(r'\'\'\'(.*?)\'\'\'|\"\"\"(.*?)\"\"\"', '', code, flags=re.DOTALL)
    # Remove empty lines
    code = re.sub(r'\n\s*\n', '\n', code)
    # Strip leading/trailing whitespace from each line
    code = '\n'.join(line.strip() for line in code.splitlines() if line.strip())
    return code
def remove_comments_and_empty_lines_JS(code):
    # Remove single-line comments
    code = re.sub(r'//.*', '', code)
    # Remove multi-line comments
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    # Remove empty lines
    code = re.sub(r'\n\s*\n', '\n', code)
    # Strip leading/trailing whitespace from each line
    code = '\n'.join(line.strip() for line in code.splitlines() if line.strip())
    return code
def count_pure_code_lines(code, lang):
    if lang == 'py':
        return len(remove_comments_and_empty_lines_PY(code).splitlines())
    if lang == 'js':
        return len(remove_comments_and_empty_lines_JS(code).splitlines())

In [4]:
code_files = [
    'web/app/admin.py',
    'web/app/apps.py',
    'web/app/utils.py',
    'web/app/consumers.py',
    'web/app/forms.py',
    'web/app/models.py',
    'web/app/routing.py',
    'web/app/urls.py',
    'web/app/views.py',
    'web/app/assistant/emotional_journal.py',
    'web/app/assistant/helpers.py',
    'web/app/assistant/moderation.py',
    'web/app/assistant/responder.py',
    'web/app/assistant/settings.py',
    'web/app/assistant/tools.py',
    'web/app/assistant/recommender.py',
    'web/app/assistant/user_profile.py',
    'web/app/assistant/Tools/tools_functions.py'
    ]

js_code_files = [
    'web/app/static/app/js/chat.js',
    'web/app/static/app/js/base.js',
    'web/app/static/app/js/balance.js'
]
code_lines = 0
for code_file in code_files:
    with open(code_file, 'r', encoding='utf-8') as file:
        code_lines += count_pure_code_lines(file.read(), 'py')

code_lines_js = 0
for code_file in js_code_files:
    with open(code_file, 'r', encoding='utf-8') as file:
        code_lines_js += count_pure_code_lines(file.read(), 'js')
    
print(f"py: {code_lines}\njs: {code_lines_js}\ntotal: {code_lines + code_lines_js}")

py: 2365
js: 719
total: 3084


#### Recommendations utils

In [3]:
from web.app.assistant.recommender import *
from dotenv import load_dotenv
import os, openai, textwrap

load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")

recommender = Recommender(
    gpt_model="gpt-4o-mini",
    recommendations_tree_path="web/app/assistant/Recommendations/recomendation_tree.json",
    recommendations_path="web/app/assistant/Recommendations/recommendations.txt"
    )

In [ ]:
# Generate recommendation
new_rec, usage = await recommender.generate_recommendation('about hygeene')
print(textwrap.fill(new_rec, width=70), f"\nPrompt tokens:{usage.prompt_tokens}\nCompletion tokens:{usage.completion_tokens}")

Incorporate a daily self-care routine that includes regular
handwashing, oral hygiene, and skin care practices to promote both
physical health and mental well-being. Consider mindful rituals, such
as using essential oils or calming music during your routines, to
enhance relaxation and self-connection. 
Prompt tokens:81
Completion tokens:55


In [ ]:
# Generate vector to it
categories = [
    "Sport", "Nutrition","Sleep", 
    "Stress", "Mental Health", "Social", 
    "Self-Care", "Motivation", "Resilience", 
    "Body", "Mindfulness", "Positive Thinking"
]

vector, usage = await recommender.generate_categories_v(categories, new_rec)
for i,categ in enumerate(list(zip(categories, vector))):
    if (i+1)%3!=0: print(f"{categ[0]}: {categ[1]}",end='; ')
    else: print(f"{categ[0]}: {categ[1]}")
print(f"Prompt tokens:{usage.prompt_tokens}\nCompletion tokens:{usage.completion_tokens}")

Sport: 2; Nutrition: 3; Sleep: 0
Stress: 0; Mental Health: 2; Social: 0
Self-Care: 8; Motivation: 0; Resilience: 0
Body: 2; Mindfulness: 3; Positive Thinking: 4
Prompt tokens:305
Completion tokens:23


In [2]:
# Save it to file
recommender.save_recomendation(vector, new_rec)

NameError: name 'vector' is not defined

In [ ]:
# Build save and tree
recommender.build_and_save_rec_tree()

0:  v: [0, 0, 0, 8, 7, 0, 0, 0, 0, 0, 0, 0] i:0
1:  v: [0, 0, 0, 7, 0, 0, 0, 0, 0, 8, 9, 7] i:4
2:  v: [0, 0, 0, 8, 7, 0, 0, 0, 0, 0, 10, 9] i:5
3:  v: [0, 0, 0, 0, 8, 0, 0, 9, 0, 0, 0, 0] i:6
4:  v: [0, 0, 0, 0, 0, 0, 6, 5, 0, 10, 10, 8] i:7
5:  v: [0, 0, 0, 0, 0, 0, 8, 6, 0, 9, 9, 8] i:8 <----- median
6:  v: [0, 3, 0, 0, 5, 0, 8, 0, 0, 3, 6, 4] i:9
7:  v: [2, 1, 0, 5, 0, 0, 2, 0, 0, 2, 4, 3] i:2
8:  v: [3, 2, 0, 1, 0, 0, 2, 2, 2, 3, 4, 5] i:3
9:  v: [6, 3, 0, 4, 5, 0, 0, 0, 0, 0, 2, 4] i:1
Depth:  0 
 __________
0:  v: [0, 0, 0, 8, 7, 0, 0, 0, 0, 0, 0, 0] i:0
1:  v: [0, 0, 0, 7, 0, 0, 0, 0, 0, 8, 9, 7] i:4
2:  v: [0, 0, 0, 8, 7, 0, 0, 0, 0, 0, 10, 9] i:5 <----- median
3:  v: [0, 0, 0, 0, 8, 0, 0, 9, 0, 0, 0, 0] i:6
4:  v: [0, 0, 0, 0, 0, 0, 6, 5, 0, 10, 10, 8] i:7
Depth:  1 
 __________
0:  v: [0, 0, 0, 8, 7, 0, 0, 0, 0, 0, 0, 0] i:0
1:  v: [0, 0, 0, 7, 0, 0, 0, 0, 0, 8, 9, 7] i:4 <----- median
Depth:  2 
 __________
0:  v: [0, 0, 0, 8, 7, 0, 0, 0, 0, 0, 0, 0] i:0 <----- median
Depth

In [13]:
from cryptography.fernet import Fernet
f = Fernet('Fi71axfcVesIV1YrZeLMaHPPkc5KLZlGGCkj1aduuyo=')
a = 'gAAAAABnWUmD6ydBLaWTnUqliauVlS83sbrO0q5CPa_2ZiBahpOLJUJ3IRmb69vOJEQQInXundgZm0J0jPT0Ax8hpC1KaZjQieKev2BAmCGDH4UQHyYIKh3W9tWW7SQagLeCzBU4CYBT_toNUCA8BpjBktO4G6UE5r1GbV64M2s4saQUzJvSL8pdSSO4vlBk6cDlxXaeBfPZLcCqkiTgx7d8coX5nWZ1g24whdPmolTM_Jahq154UXhRd74dYx7THK1XuCCdEjtfarA9KqopQ8WdhBWlpw15gIj9t3-7sJx0FER7uzU2fNlCnLuDEfkdZFkrtX3V_yH_sMF5tsNgwGEh4uUjE8lfEy-KUgpObAbekdaYWw-iAxMVIrurzIY_nGXrG40WMUfpJ9mC7D_YWDJmzqUt_L-WCJ5FknobicTv3D8Zr8SkzqM='
f.encrypt(b'[]')

b'gAAAAABnWZeAWTDPxQd713dXqu6JKKmIOspM14mfOGswnU_JKPKxBqg5FTkn3nRpp8A6vCOiY8ORbCU9xnkIwbCbLfeld1zyxA=='

In [12]:
def encripted_data_len(text_len):
    if text_len < 16:
        return 100
    data_len = 100
    for i in range(text_len//16):
        if i%3==0: data_len += 24
        else: data_len +=20
    return data_len
encripted_data_len(10000)

13436

In [27]:
a = []
a[-1:]

[]

In [114]:
from cryptography.fernet import Fernet

a = Fernet.generate_key().decode()
f = Fernet(a)
t = ''
fir = len(f.encrypt(t.encode()))
for i in range(300):
    t += 's'
    if len(f.encrypt(t.encode())) != fir:
        fir = len(f.encrypt(t.encode()))
        print(len(f.encrypt(t.encode())), len(t))

120 16
140 32
164 48
184 64
204 80
228 96
248 112
268 128
292 144
312 160
332 176
356 192
376 208
396 224
420 240
440 256
460 272
484 288


AttributeError: 'list' object has no attribute 'item'